In [1]:
import os, sys, json
sys.path.append(os.path.realpath(f"{os.getcwd()}/../../"))

from autocog.pm.sta.backend import Backend
from autocog.pm.sta.syntax import Syntax
from autocog.pm.sta.automaton import Automaton as STA
from autocog.utility.pynb import wrap_graphviz, display

In [2]:
sources = ["""
prompt mmlu_main {
    is {
        topic is text<20>;
        question is text<50>;
        choices[4] is {
            value is text<40>;
            correct is enum("yes","no");
        }
        answer is repeat(.choices.value);
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
    }
    return {
        from .answer;
    }
    annotate {
        _ as "You are answering a multiple choice questionnaire.";
        .topic           as "the general category from which the question was taken";
        .question        as "the question that you have to answer";
        .choices         as "you judge whether each choice is correct or not";
        .choices.value   as "the value of the choice";
        .choices.correct as "you decide whether this choice is correct or not";
        .answer          as "you repeat the value of the choice that best answer the question";
    }
}""","""
format bool {
    is enum("true","false");
    annotate "a bolean value is either true or false";
}

format sentence {
    argument length=50;
    argument language="english";
    is text<length=$length>;
    annotate f"a grammatically correct {language} sentence of at most {length} tokens";
}

format response {
    is repeat(.choices.value);
    annotate f"repeat verbatim the value of the correct choice";
}

struct a_choice {
    is {
        value   is sentence<40>;
        correct is bool;
    }
    annotate {
        .value   as "the value of the choice";
        .correct as "you decide whether this choice is correct or not";
    }
}

prompt mmlu_main {
    is {
        topic      is text<20>;
        question   is sentence;
        choices[4] is a_choice;
        answer     is response;
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
    }
    return .answer;
    annotate {
        _ as "You are answering a multiple choice questionnaire.";
        .topic    as "the general category from which the question was taken";
        .question as "the question that you have to answer";
        .choices  as "you judge whether each choice is correct or not";
        .answer   as "you repeat the value of the choice that best answer the question";
    }
}
"""]
sources = ["""
format bool {
    is enum("true","false");
}
prompt mmlu_main {
    is {
        A is bool;
        B[0:2] is {
            C is bool;
            D[2:4] is bool;
        }
        E[0:2] is {
          F is bool;
        }
        G is {
          H is bool;
          I is bool;
        }
        J is bool;
    }
}""","""
format bool {
    is enum("true","false");
}
prompt mmlu_main {
    is {
        A[0:2] is {
            B is bool;
        }
        C[0:3] is bool;
        D is bool;
    }
}"""]
syntax = Syntax(header_pre_post=('[INST]\n<<SYS>>You are an expert interacting with the world using an interactive prompting system.<</SYS>>','[/INST]'))

In [3]:
for source in sources:
    backend = Backend(source=source)
    for (name, prompt) in backend.program.prompts.items():
        sta = STA(prompt=prompt)

        sta.build_abstract()
        # display(wrap_graphviz(sta.toGraphViz_abstract()))

        sta.build_concrete_v1()
        display(wrap_graphviz(sta.toGraphViz_concrete()))

        # fta = sta.instantiate(syntax, data={'A':True,'E':[{'F':False},{}]})
        # display(wrap_graphviz(fta.toGraphViz(label_with_uid=True)))